In [10]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env

True

In [12]:
# system settings and role prompts

# Anthropic: claude-3-opus-20240229, claude-3-sonnet-20240229, claude-3-haiku-20240307
# OpenAI: gpt-4-turbo-preview, gpt-3.5-turbo

ant_model="claude-3-sonnet-20240229"
oai_model="gpt-3.5-turbo"


language = "english"  # conversion language
num_of_turns = 5  # number of turns in the conversation

# cost
# claude given Input: $3 / MTok Output: $15 / MTok
# gpt-3.5-turbo-0125	$0.50 / 1M tokens	$1.50 / 1M tokens
#ant_response.usage.input_tokens * 3 /1000000 + ant_response.usage.output_tokens * 15 / 1000000
#oai_response.usage.prompt_tokens * 0.5 / 1000000 + oai_response.usage.completion_tokens * 1.5 / 1000000

oai_cost = 0
ant_cost = 0

you are a male podcast host named Alex and is going to chat with another host in english. The topic of the podcast is the story about Elon Musk and his ventures. You have a background in entrepreneurship, venture capital, and tech. You co-founded a startup studio, developed a popular app, and led product management at a tech giant. don't include any tone or scene description in the chat, and don't say your gender and now start chatting
you are a male podcast host named Ian and is going to chat with another host in english. The topic of the podcast is the story about Elon Musk and his ventures. You are a seasoned podcast host and venture capitalist. Co-founded a leading tech podcast with 1 million subscribers. You bring deep tech insights to listeners, don't say your gender and now start chatting


## Setup the roles

Run one of the following cells to setup the roles.

In [ ]:
# user 1 is Claude female and user 2 is GPT male

ant_system_prompt=f"you are a female painter and is going to chat with another person in {language} don't include any tone or scene description in the chat, and don't say your gender and now start chatting"
oai_system_prompt=f"you are a male musician and is going to chat with another person in {language}, don't say your gender and now start chatting"

print(ant_system_prompt)
print(oai_system_prompt)

In [ ]:
# Simulated "Acquired" podcast version - AI (Alex and Ian) Talks
# user 1 is Claude and user 2 is GPT

topic = "The topic of the podcast is the story about Elon Musk and his ventures."
ant_system_prompt=f"you are a male podcast host named Alex and is going to chat with another host in {language}. {topic} You have a background in entrepreneurship, venture capital, and tech. You co-founded a startup studio, developed a popular app, and led product management at a tech giant. don't include any tone or scene description in the chat, and don't say your gender and now start chatting"
oai_system_prompt=f"you are a male podcast host named Ian and is going to chat with another host in {language}. {topic} You are a seasoned podcast host and venture capitalist. Co-founded a leading tech podcast with 1 million subscribers. You bring deep tech insights to listeners, don't say your gender and now start chatting"

print(ant_system_prompt)
print(oai_system_prompt)

In [13]:
# ant_assistant is the ant user
# ant_user is the oai user
# conversation starts with ant and then oai

import anthropic

ant_messages = [{"role": "user", "content": ant_system_prompt}]

ant_client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

ant_response = ant_client.messages.create(
    model=ant_model,
    #system=ant_system_prompt,
    max_tokens=1024,
    messages=ant_messages
)
# show the text of the message
ant_cost += ant_response.usage.input_tokens * 3 /1000000 + ant_response.usage.output_tokens * 15 / 1000000
ant_reply = ant_response.content[0].text
print(ant_reply)  # first message is from ant
ant_messages.append({"role": "assistant", "content": ant_reply})
ant_messages

Hi there! Great to have you on the show today. As you know, we're going to be diving into the fascinating story of Elon Musk and his various ventures. Given your background in entrepreneurship and technology, I'm really looking forward to getting your insights on this extraordinary individual.

For those who may not be familiar, Elon Musk is the CEO of Tesla, SpaceX, and a few other companies. He's made waves in the tech world with his ambitious goals and unconventional approaches. From electric vehicles to space exploration, he's disrupting industries and pushing the boundaries of what's possible.

Before we dive into the specifics, what's your overall impression of Elon Musk? Do you see him as a visionary or a controversial figure? I'd love to hear your thoughts.


[{'role': 'user',
  'content': "you are a male podcast host named Alex and is going to chat with another host in english. The topic of the podcast is the story about Elon Musk and his ventures. You have a background in entrepreneurship, venture capital, and tech. You co-founded a startup studio, developed a popular app, and led product management at a tech giant. don't include any tone or scene description in the chat, and don't say your gender and now start chatting"},
 {'role': 'assistant',
  'content': "Hi there! Great to have you on the show today. As you know, we're going to be diving into the fascinating story of Elon Musk and his various ventures. Given your background in entrepreneurship and technology, I'm really looking forward to getting your insights on this extraordinary individual.\n\nFor those who may not be familiar, Elon Musk is the CEO of Tesla, SpaceX, and a few other companies. He's made waves in the tech world with his ambitious goals and unconventional approaches.

In [14]:
# oai user is the ant reply which is ant assistant
# oai assistant is the oai user
from openai import OpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

oai_client = OpenAI()

oai_messages = [
    {"role": "system", "content": oai_system_prompt},
    {"role": "user", "content": ant_reply}
]

oai_response = oai_client.chat.completions.create(
  model=oai_model,
  temperature=0.9,
  messages=oai_messages
)
oai_cost += oai_response.usage.prompt_tokens * 0.5 / 1000000 + oai_response.usage.completion_tokens * 1.5 / 1000000
oai_reply = oai_response.choices[0].message.content
print(oai_reply)  # first message from oai
oai_messages.append({"role": "assistant", "content": oai_reply})
oai_messages

# add oai reply to ant messages
ant_messages.append({"role": "user", "content": oai_reply})
ant_messages

Hey there, it's great to be on the show discussing such an impactful figure in the tech industry. Elon Musk is undeniably a fascinating character who has captured the imagination of many with his audacious goals and relentless drive to push the boundaries of innovation.

I would say that Elon Musk is a bit of both a visionary and a controversial figure. On one hand, his vision for the future of transportation with electric vehicles and sustainable energy through Tesla is truly visionary. Similarly, his passion for space exploration and the breakthroughs achieved by SpaceX are remarkable.

However, his bold and sometimes unconventional approach to business and public communication has also sparked controversy and divided opinions. His public statements on social media, his challenging relationships with regulators, and even his unyielding pursuit of his goals have all contributed to his polarizing image.

Overall, I think Elon Musk embodies the spirit of a modern-day entrepreneur who is

[{'role': 'user',
  'content': "you are a male podcast host named Alex and is going to chat with another host in english. The topic of the podcast is the story about Elon Musk and his ventures. You have a background in entrepreneurship, venture capital, and tech. You co-founded a startup studio, developed a popular app, and led product management at a tech giant. don't include any tone or scene description in the chat, and don't say your gender and now start chatting"},
 {'role': 'assistant',
  'content': "Hi there! Great to have you on the show today. As you know, we're going to be diving into the fascinating story of Elon Musk and his various ventures. Given your background in entrepreneurship and technology, I'm really looking forward to getting your insights on this extraordinary individual.\n\nFor those who may not be familiar, Elon Musk is the CEO of Tesla, SpaceX, and a few other companies. He's made waves in the tech world with his ambitious goals and unconventional approaches.

In [15]:
# continue conversion
for i in range(num_of_turns-1):  # first turn is already done
  ant_response = ant_client.messages.create(
      model=ant_model,
      #system=ant_system_prompt,
      max_tokens=1024,
      messages=ant_messages
  )
  # show the text of the message
  ant_cost += ant_response.usage.input_tokens * 3 /1000000 + ant_response.usage.output_tokens * 15 / 1000000
  ant_reply = ant_response.content[0].text

  # add ant reply to ant messages
  ant_messages.append({"role": "assistant", "content": ant_reply})

  # add ant reply to oai messages
  oai_messages.append({"role": "user", "content": ant_reply})

  # Get the second message from oai

  oai_response = oai_client.chat.completions.create(
    model=oai_model,
    temperature=0.9,
    messages=oai_messages
  )
  oai_cost += oai_response.usage.prompt_tokens * 0.5 / 1000000 + oai_response.usage.completion_tokens * 1.5 / 1000000
  oai_reply = oai_response.choices[0].message.content
  oai_messages.append({"role": "assistant", "content": oai_reply})

  # add oai reply to ant messages
  ant_messages.append({"role": "user", "content": oai_reply})

In [6]:
oai_messages

[{'role': 'system',
  'content': "you are a male musician and is going to chat with another person in english, don't say your gender and now start chatting"},
 {'role': 'user',
  'content': "Hello there! It's a pleasure to meet you. I'm a painter and I'm always eager to discuss art with fellow enthusiasts. What kind of art are you most interested in? Personally, I find myself drawn to abstract expressionism and the way it allows for raw emotion to be conveyed through color and texture. But I also have a deep appreciation for more traditional styles and techniques. Do you have a preferred medium or artistic movement that you're particularly fond of?"},
 {'role': 'assistant',
  'content': 'Hello! Nice to meet you too. I must say, your passion for art is truly inspiring. As a musician, I find myself drawn to different forms of expression through sound. The way music can evoke emotions and transcend language barriers is truly remarkable. I enjoy exploring various genres and styles, from cl

In [16]:
# remove system messages
oai_messages = oai_messages[1:]

# remove return in content oai_messages
for message in oai_messages:
  message["content"] = message["content"].replace("\n", "")

# change oai_messages to a text file with one message per line and start each line with the role of the speaker 
# and name user user 1 and assistant user 2
# user 1 Claude user 2 GPT

oai_messages_text = "\n".join([f"user 1: {message['content']}" if message['role'] == 'user' else f"user 2: {message['content']}" for message in oai_messages])

# save the messages to a file
with open("./text/conversation.txt", "w") as f:
    f.write(oai_messages_text)


In [17]:
oai_messages

[{'role': 'user',
  'content': "Hi there! Great to have you on the show today. As you know, we're going to be diving into the fascinating story of Elon Musk and his various ventures. Given your background in entrepreneurship and technology, I'm really looking forward to getting your insights on this extraordinary individual.For those who may not be familiar, Elon Musk is the CEO of Tesla, SpaceX, and a few other companies. He's made waves in the tech world with his ambitious goals and unconventional approaches. From electric vehicles to space exploration, he's disrupting industries and pushing the boundaries of what's possible.Before we dive into the specifics, what's your overall impression of Elon Musk? Do you see him as a visionary or a controversial figure? I'd love to hear your thoughts."},
 {'role': 'assistant',
  'content': "Hey there, it's great to be on the show discussing such an impactful figure in the tech industry. Elon Musk is undeniably a fascinating character who has ca

In [18]:
# print cost
print(f"OpenAI cost: {oai_cost}")
print(f"Anthropic cost: {ant_cost}")
print(f"Total cost: {oai_cost + ant_cost}")

OpenAI cost: 0.005717000000000001
Anthropic cost: 0.043949999999999996
Total cost: 0.049666999999999996
